<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/dataset/txt_to_json_MultihopRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json

# txt 파일 경로와 생성할 json 파일 경로 설정
input_txt_file = '/content/유형4_v2.txt'  # 질문들이 저장된 텍스트 파일 경로
output_json_file = '/content/questions4.json'  # 결과 JSON 파일 경로

# 텍스트 파일에서 질문들을 읽어오기
with open(input_txt_file, 'r', encoding='utf-8') as f:
    questions = f.read().splitlines()

# 각 질문을 JSON 형식으로 변환
json_data = []
for question in questions:
    if question.strip():  # 빈 줄 무시
        json_data.append({
            "type": "4",
            "dataset": "Multihop-RAG",
            "question": question.strip()
        })

# JSON 파일로 저장
with open(output_json_file, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

print("Questions have been saved to", output_json_file)


Questions have been saved to /content/questions4.json


In [ ]:
import json

# 파일 경로 설정
questions_file = '/content/questions1.json'  # questions1.json 파일 경로
multihop_file = '/content/MultiHopRAG.json'  # Multihop-RAG.json 파일 경로
output_file = '/content/multihop_class.json'  # 최종 결과 파일 경로

# questions1.json 파일 읽기
with open(questions_file, 'r', encoding='utf-8') as f:
    questions_data = json.load(f)

# Multihop-RAG.json 파일 읽기
with open(multihop_file, 'r', encoding='utf-8') as f:
    multihop_data = json.load(f)

# Multihop-RAG 데이터를 query 기준으로 검색 가능하게 변환
query_to_data = {item['query']: item for item in multihop_data}

# 매칭하여 answer와 evidence_list 추가 또는 answer에 "helpme" 추가
for question in questions_data:
    query_text = question['question']
    # query 필드와 일치하는 항목이 있으면 answer와 evidence_list 추가
    if query_text in query_to_data:
        match_data = query_to_data[query_text]
        question['answer'] = match_data['answer']
        question['evidence_list'] = match_data['evidence_list']
    else:
        # 일치하는 query가 없을 경우 answer에 "helpme" 추가
        question['answer'] = "helpme"
        question['evidence_list'] = []

# 결과 JSON 파일로 저장
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(questions_data, f, ensure_ascii=False, indent=4)

print("The merged data has been saved to", output_file)


The merged data has been saved to /content/multihop_class.json


In [ ]:
!pip install openai
!pip install -q groq
!pip install -U accelerate bitsandbytes datasets evaluate
!pip install -U peft transformers trl
import openai
import time
from tqdm import tqdm
!pip install openai==0.28
import json

In [ ]:
from google.colab import userdata, drive
# For Hugging Face Hub setting
from huggingface_hub import login

# Upload the HuggingFace token (should have WRITE access) from Colab secrets
HF = userdata.get('HF')

# This is needed to upload the model to HuggingFace
login(token=HF,add_to_git_credential=True)
import openai
from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# 질문에서 엔터티를 추출하는 함수
def get_entities(query):
    messages = [
        {
            "role": "system",
            "content": """
            Extract all the named entities from the following question. Don't print anything other than the entities.
            ex)
            question: What was the father of Kasper Schmeichel voted to be by the IFFHS in 1992?
            ->
            Kasper Schmeichel,IFFHS,1992
            """
        },
        {
            "role": "user",
            "content": query
        }
    ]
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=messages
    )
    return response.choices[0].message['content']

# JSON 파일 경로 설정
input_file = '/content/multihop_class.json'  # 기존 JSON 파일 경로
output_file = '/content/multihop_class_with_context_url.json'  # 결과 JSON 파일 경로

# 파일 읽기
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터 변환 및 context_url 추가
new_data = []
for entry in data:
    question = entry['question']
    entities = get_entities(question)  # LLM으로부터 엔터티 추출
    entity_list = [entity.strip() for entity in entities.split(',')]  # 엔터티를 리스트로 변환

    # 엔터티를 기반으로 context_url 생성
    context_url = [f"https://en.wikipedia.org/wiki/{entity.replace(' ', '_')}" for entity in entity_list]

    # 새로운 엔트리 구성
    new_entry = {
        "type": entry['type'],
        "dataset": entry['dataset'],
        "question": question,
        "context_url": context_url,
        "answer": entry.get('answer', "helpme"),  # 기존 answer를 가져오거나 없으면 'helpme'
        "evidence_list": entry.get('evidence_list', [])  # 기존 evidence_list를 가져오거나 빈 리스트
    }
    new_data.append(new_entry)

# 결과 저장
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(new_data, f, ensure_ascii=False, indent=4)

print("The data with context_url has been saved to", output_file)

The data with context_url has been saved to /content/multihop_class_with_context_url.json


In [ ]:
###################################
#널쿼리 제거

In [ ]:
import json

# 파일 경로 설정
input_file = '/content/multihopRAG_4type_classification_urls_answer_v2.json'  # 원본 JSON 파일 경로
output_file_filtered = '/content/fMultiHopRAG_4type_classification_500_v3_ffff.json'  # "Insufficient information."을 제외한 데이터 저장 파일 경로
output_file_insufficient = '/content/fMultiHopRAG_4type_classification_500_nullquery_ffff.json'  # "Insufficient information." 데이터 저장 파일 경로

# 원본 파일 읽기
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 데이터를 분류할 리스트
filtered_data = []         # "Insufficient information."이 아닌 데이터
insufficient_info_data = []  # "Insufficient information."인 데이터

# 데이터를 조건에 따라 분리
for entry in data:
    if entry.get('answer') == "Insufficient information.":
        insufficient_info_data.append(entry)  # "Insufficient information."인 데이터 추가
    else:
        filtered_data.append(entry)  # 나머지 데이터 추가

# 분리된 데이터 저장
with open(output_file_filtered, 'w', encoding='utf-8') as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

with open(output_file_insufficient, 'w', encoding='utf-8') as f:
    json.dump(insufficient_info_data, f, ensure_ascii=False, indent=4)

print("Filtered data has been saved to", output_file_filtered)
print("Insufficient information data has been saved to", output_file_insufficient)


Filtered data has been saved to /content/fMultiHopRAG_4type_classification_500_v3_ffff.json
Insufficient information data has been saved to /content/fMultiHopRAG_4type_classification_500_nullquery_ffff.json
